In [26]:
import pandas as pd

import tabula

In [27]:
#table = tabula.read_pdf('papers/haunschild2012.pdf',pages=4)
table = tabula.read_pdf('../data/haunschild_g2/haunschild2012.pdf',pages=4)

df = table[0]

df.columns= df.loc[0]


part1 = df.iloc[1:,[0,1,6]]
part2 = df.iloc[1:,[10,11,16]]

part1.columns = ['Formula','Name','E']
part2.columns = ['Formula','Name','E']

df1 = pd.concat([part1,part2]).reset_index(drop=True)

In [28]:
#table = tabula.read_pdf('papers/haunschild2012.pdf',pages=5)
table = tabula.read_pdf('../data/haunschild_g2/haunschild2012.pdf',pages=5)

df = table[0]

df.columns= df.loc[0]

df = df.iloc[1:,[0,1,6]]

df2 = df.dropna().iloc[:-4,:]

Got stderr: Jun 22, 2022 6:34:28 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



In [29]:
df2.columns = ['Formula','Name','E']
df = pd.concat([df1.iloc[1:],df2])

In [30]:
df = df.reset_index(drop=True)

In [31]:
df['Formula'] = df['Formula'].apply(lambda x: x.replace(' ',''))

In [32]:
df[~df['Formula'].apply(lambda x: len(x) < 10)]

,Formula,Name,E
130,"valuesfromRef.43,theW4valuesfromRef.9,andtheG2...",Dioxygen,505.88
142,leveloftheoryis0.47kJ/mollowerthanforthestruct...,Triplet silylene,555.72
144,SiHmulti-referenceeffectsinsingletCH2arenotsuf...,Silane,1357.91


In [33]:
df.loc[130,'Formula'] = 'O2'
df.loc[142,'Formula'] = 'SiH2'
df.loc[144,'Formula'] = 'SiH4'
df.loc[132,'Formula'] = 'O3'

In [34]:
from itertools import permutations
from ase.io import read, write
import numpy as np 
from ase import Atoms

In [35]:
def expand_molname(name):
    import re
    def it(l):
        if not isinstance(l,list):
            l = [l]
        for l_ in l:
            yield l_ 
    
    # Splitting on UpperCase using re
    res_list = []
    res_list = re.findall('[A-Z][^A-Z]*', name)
    for i, a in enumerate(res_list):
        if a[-1].isnumeric():
            res_list[i] = [a[:-1]]*int(a[-1])
    res_list = [r for l in res_list for r in it(l)]
    return res_list

In [39]:
df['xyz_idx']= [[] for _ in range(len(df))]

In [40]:
for a_idx, a in enumerate(read('../data/haunschild_g2/g2_97.xyz',':')):
    for loc, itm in df.iterrows():
        form = itm.Formula
        try:
            df_num = np.sort(Atoms(form).get_atomic_numbers())
            print(a_idx, a, loc, form, df_num)
        except ValueError:
            print(loc, form)
            raise ValueError
        a_num = np.sort(a.get_atomic_numbers())
        if len(df_num) == len(a_num) and np.allclose(df_num, a_num):
            df.loc[loc, 'xyz_idx'].append(a_idx)

0 Atoms(symbols='OC', pbc=False) 0 AlCl3 [13 17 17 17]
0 Atoms(symbols='OC', pbc=False) 1 AlF3 [ 9  9  9 13]
0 Atoms(symbols='OC', pbc=False) 2 BCl3 [ 5 17 17 17]
0 Atoms(symbols='OC', pbc=False) 3 BF3 [5 9 9 9]
0 Atoms(symbols='OC', pbc=False) 4 BeH [1 4]
0 Atoms(symbols='OC', pbc=False) 5 CCl4 [ 6 17 17 17 17]
0 Atoms(symbols='OC', pbc=False) 6 CF4 [6 9 9 9 9]
0 Atoms(symbols='OC', pbc=False) 7 CH [1 6]
0 Atoms(symbols='OC', pbc=False) 8 CH2Cl2 [ 1  1  6 17 17]
0 Atoms(symbols='OC', pbc=False) 9 CH2F2 [1 1 6 9 9]
0 Atoms(symbols='OC', pbc=False) 10 CH2O2 [1 1 6 8 8]
0 Atoms(symbols='OC', pbc=False) 11 CH2O [1 1 6 8]
0 Atoms(symbols='OC', pbc=False) 12 CH2 [1 1 6]
0 Atoms(symbols='OC', pbc=False) 13 CH2 [1 1 6]
0 Atoms(symbols='OC', pbc=False) 14 CH3Cl [ 1  1  1  6 17]
0 Atoms(symbols='OC', pbc=False) 15 CH3 [1 1 1 6]
0 Atoms(symbols='OC', pbc=False) 16 CH3O [1 1 1 6 8]
0 Atoms(symbols='OC', pbc=False) 17 CH3O [1 1 1 6 8]
0 Atoms(symbols='OC', pbc=False) 18 CH3S [ 1  1  1  6 16]
0 Ato

6 Atoms(symbols='SH', pbc=False) 33 C2H2 [1 1 6 6]
6 Atoms(symbols='SH', pbc=False) 34 C2H2O2 [1 1 6 6 8 8]
6 Atoms(symbols='SH', pbc=False) 35 C2H2O [1 1 6 6 8]
6 Atoms(symbols='SH', pbc=False) 36 C2H3Cl [ 1  1  1  6  6 17]
6 Atoms(symbols='SH', pbc=False) 37 C2H3 [1 1 1 6 6]
6 Atoms(symbols='SH', pbc=False) 38 C2H3F [1 1 1 6 6 9]
6 Atoms(symbols='SH', pbc=False) 39 C2H3O [1 1 1 6 6 8]
6 Atoms(symbols='SH', pbc=False) 40 C2H3OCl [ 1  1  1  6  6  8 17]
6 Atoms(symbols='SH', pbc=False) 41 C2H3OF [1 1 1 6 6 8 9]
6 Atoms(symbols='SH', pbc=False) 42 C2H4 [1 1 1 1 6 6]
6 Atoms(symbols='SH', pbc=False) 43 C2H4O2 [1 1 1 1 6 6 8 8]
6 Atoms(symbols='SH', pbc=False) 44 C2H4O2 [1 1 1 1 6 6 8 8]
6 Atoms(symbols='SH', pbc=False) 45 C2H4O [1 1 1 1 6 6 8]
6 Atoms(symbols='SH', pbc=False) 46 C2H4O [1 1 1 1 6 6 8]
6 Atoms(symbols='SH', pbc=False) 47 C2H4S [ 1  1  1  1  6  6 16]
6 Atoms(symbols='SH', pbc=False) 48 C2H5Cl [ 1  1  1  1  1  6  6 17]
6 Atoms(symbols='SH', pbc=False) 49 C2H5 [1 1 1 1 1 6 6]


12 Atoms(symbols='SCS', pbc=False) 71 C3H7 [1 1 1 1 1 1 1 6 6 6]
12 Atoms(symbols='SCS', pbc=False) 72 C3H8O [1 1 1 1 1 1 1 1 6 6 6 8]
12 Atoms(symbols='SCS', pbc=False) 73 C3H8O [1 1 1 1 1 1 1 1 6 6 6 8]
12 Atoms(symbols='SCS', pbc=False) 74 C3H8 [1 1 1 1 1 1 1 1 6 6 6]
12 Atoms(symbols='SCS', pbc=False) 75 C3NH3 [1 1 1 6 6 6 7]
12 Atoms(symbols='SCS', pbc=False) 76 C3NH9 [1 1 1 1 1 1 1 1 1 6 6 6 7]
12 Atoms(symbols='SCS', pbc=False) 77 C4H10 [1 1 1 1 1 1 1 1 1 1 6 6 6 6]
12 Atoms(symbols='SCS', pbc=False) 78 C4H10 [1 1 1 1 1 1 1 1 1 1 6 6 6 6]
12 Atoms(symbols='SCS', pbc=False) 79 C4H4O [1 1 1 1 6 6 6 6 8]
12 Atoms(symbols='SCS', pbc=False) 80 C4H4S [ 1  1  1  1  6  6  6  6 16]
12 Atoms(symbols='SCS', pbc=False) 81 C4H6 [1 1 1 1 1 1 6 6 6 6]
12 Atoms(symbols='SCS', pbc=False) 82 C4H6 [1 1 1 1 1 1 6 6 6 6]
12 Atoms(symbols='SCS', pbc=False) 83 C4H6 [1 1 1 1 1 1 6 6 6 6]
12 Atoms(symbols='SCS', pbc=False) 84 C4H6 [1 1 1 1 1 1 6 6 6 6]
12 Atoms(symbols='SCS', pbc=False) 85 C4H6 [1 1 1 1

18 Atoms(symbols='CNH', pbc=False) 99 CS2 [ 6 16 16]
18 Atoms(symbols='CNH', pbc=False) 100 FCl [ 9 17]
18 Atoms(symbols='CNH', pbc=False) 101 F2 [9 9]
18 Atoms(symbols='CNH', pbc=False) 102 F3Cl [ 9  9  9 17]
18 Atoms(symbols='CNH', pbc=False) 103 HCl [ 1 17]
18 Atoms(symbols='CNH', pbc=False) 104 HF [1 9]
18 Atoms(symbols='CNH', pbc=False) 105 HOCl [ 1  8 17]
18 Atoms(symbols='CNH', pbc=False) 106 HO [1 8]
18 Atoms(symbols='CNH', pbc=False) 107 HS [ 1 16]
18 Atoms(symbols='CNH', pbc=False) 108 H2 [1 1]
18 Atoms(symbols='CNH', pbc=False) 109 H2O2 [1 1 8 8]
18 Atoms(symbols='CNH', pbc=False) 110 H2O [1 1 8]
18 Atoms(symbols='CNH', pbc=False) 111 H2S [ 1  1 16]
18 Atoms(symbols='CNH', pbc=False) 112 LiF [3 9]
18 Atoms(symbols='CNH', pbc=False) 113 LiH [1 3]
18 Atoms(symbols='CNH', pbc=False) 114 Li2 [3 3]
18 Atoms(symbols='CNH', pbc=False) 115 Na2 [11 11]
18 Atoms(symbols='CNH', pbc=False) 116 NaCl [11 17]
18 Atoms(symbols='CNH', pbc=False) 117 NF3 [7 9 9 9]
18 Atoms(symbols='CNH', pbc=

21 Atoms(symbols='N2O', pbc=False) 56 C2H6S [ 1  1  1  1  1  1  6  6 16]
21 Atoms(symbols='N2O', pbc=False) 57 C2N2 [6 6 7 7]
21 Atoms(symbols='N2O', pbc=False) 58 C2NF3 [6 6 7 9 9 9]
21 Atoms(symbols='N2O', pbc=False) 59 C2NH3 [1 1 1 6 6 7]
21 Atoms(symbols='N2O', pbc=False) 60 C2NH5 [1 1 1 1 1 6 6 7]
21 Atoms(symbols='N2O', pbc=False) 61 C2NH5O [1 1 1 1 1 6 6 7 8]
21 Atoms(symbols='N2O', pbc=False) 62 C2NH7 [1 1 1 1 1 1 1 6 6 7]
21 Atoms(symbols='N2O', pbc=False) 63 C2NH7 [1 1 1 1 1 1 1 6 6 7]
21 Atoms(symbols='N2O', pbc=False) 64 C3H4 [1 1 1 1 6 6 6]
21 Atoms(symbols='N2O', pbc=False) 65 C3H4 [1 1 1 1 6 6 6]
21 Atoms(symbols='N2O', pbc=False) 66 C3H4 [1 1 1 1 6 6 6]
21 Atoms(symbols='N2O', pbc=False) 67 C3H6 [1 1 1 1 1 1 6 6 6]
21 Atoms(symbols='N2O', pbc=False) 68 C3H6O [1 1 1 1 1 1 6 6 6 8]
21 Atoms(symbols='N2O', pbc=False) 69 C3H6 [1 1 1 1 1 1 6 6 6]
21 Atoms(symbols='N2O', pbc=False) 70 C3H7Cl [ 1  1  1  1  1  1  1  6  6  6 17]
21 Atoms(symbols='N2O', pbc=False) 71 C3H7 [1 1 1 

...

134 Atoms(symbols='C2Cl4', pbc=False) 38 C2H3F [1 1 1 6 6 9]
134 Atoms(symbols='C2Cl4', pbc=False) 39 C2H3O [1 1 1 6 6 8]
134 Atoms(symbols='C2Cl4', pbc=False) 40 C2H3OCl [ 1  1  1  6  6  8 17]
134 Atoms(symbols='C2Cl4', pbc=False) 41 C2H3OF [1 1 1 6 6 8 9]
134 Atoms(symbols='C2Cl4', pbc=False) 42 C2H4 [1 1 1 1 6 6]
134 Atoms(symbols='C2Cl4', pbc=False) 43 C2H4O2 [1 1 1 1 6 6 8 8]
134 Atoms(symbols='C2Cl4', pbc=False) 44 C2H4O2 [1 1 1 1 6 6 8 8]
134 Atoms(symbols='C2Cl4', pbc=False) 45 C2H4O [1 1 1 1 6 6 8]
134 Atoms(symbols='C2Cl4', pbc=False) 46 C2H4O [1 1 1 1 6 6 8]
134 Atoms(symbols='C2Cl4', pbc=False) 47 C2H4S [ 1  1  1  1  6  6 16]
134 Atoms(symbols='C2Cl4', pbc=False) 48 C2H5Cl [ 1  1  1  1  1  6  6 17]
134 Atoms(symbols='C2Cl4', pbc=False) 49 C2H5 [1 1 1 1 1 6 6]
134 Atoms(symbols='C2Cl4', pbc=False) 50 C2H5O [1 1 1 1 1 6 6 8]
134 Atoms(symbols='C2Cl4', pbc=False) 51 C2H6 [1 1 1 1 1 1 6 6]
134 Atoms(symbols='C2Cl4', pbc=False) 52 C2H6O [1 1 1 1 1 1 6 6 8]
134 Atoms(symbols='C2C

137 Atoms(symbols='COH3NO', pbc=False) 131 O2S [ 8  8 16]
137 Atoms(symbols='COH3NO', pbc=False) 132 O3 [8 8 8]
137 Atoms(symbols='COH3NO', pbc=False) 133 P2 [15 15]
137 Atoms(symbols='COH3NO', pbc=False) 134 PF3 [ 9  9  9 15]
137 Atoms(symbols='COH3NO', pbc=False) 135 PH2 [ 1  1 15]
137 Atoms(symbols='COH3NO', pbc=False) 136 PH3 [ 1  1  1 15]
137 Atoms(symbols='COH3NO', pbc=False) 137 S2 [16 16]
137 Atoms(symbols='COH3NO', pbc=False) 138 SiCH6 [ 1  1  1  1  1  1  6 14]
137 Atoms(symbols='COH3NO', pbc=False) 139 SiCl4 [14 17 17 17 17]
137 Atoms(symbols='COH3NO', pbc=False) 140 SiF4 [ 9  9  9  9 14]
137 Atoms(symbols='COH3NO', pbc=False) 141 SiH2 [ 1  1 14]
137 Atoms(symbols='COH3NO', pbc=False) 142 SiH2 [ 1  1 14]
137 Atoms(symbols='COH3NO', pbc=False) 143 SiH3 [ 1  1  1 14]
137 Atoms(symbols='COH3NO', pbc=False) 144 SiH4 [ 1  1  1  1 14]
137 Atoms(symbols='COH3NO', pbc=False) 145 SiO [ 8 14]
137 Atoms(symbols='COH3NO', pbc=False) 146 Si2H6 [ 1  1  1  1  1  1 14 14]
137 Atoms(symbols='

140 Atoms(symbols='C2ClOH3', pbc=False) 77 C4H10 [1 1 1 1 1 1 1 1 1 1 6 6 6 6]
140 Atoms(symbols='C2ClOH3', pbc=False) 78 C4H10 [1 1 1 1 1 1 1 1 1 1 6 6 6 6]
140 Atoms(symbols='C2ClOH3', pbc=False) 79 C4H4O [1 1 1 1 6 6 6 6 8]
140 Atoms(symbols='C2ClOH3', pbc=False) 80 C4H4S [ 1  1  1  1  6  6  6  6 16]
140 Atoms(symbols='C2ClOH3', pbc=False) 81 C4H6 [1 1 1 1 1 1 6 6 6 6]
140 Atoms(symbols='C2ClOH3', pbc=False) 82 C4H6 [1 1 1 1 1 1 6 6 6 6]
140 Atoms(symbols='C2ClOH3', pbc=False) 83 C4H6 [1 1 1 1 1 1 6 6 6 6]
140 Atoms(symbols='C2ClOH3', pbc=False) 84 C4H6 [1 1 1 1 1 1 6 6 6 6]
140 Atoms(symbols='C2ClOH3', pbc=False) 85 C4H6 [1 1 1 1 1 1 6 6 6 6]
140 Atoms(symbols='C2ClOH3', pbc=False) 86 C4H8 [1 1 1 1 1 1 1 1 6 6 6 6]
140 Atoms(symbols='C2ClOH3', pbc=False) 87 C4H8 [1 1 1 1 1 1 1 1 6 6 6 6]
140 Atoms(symbols='C2ClOH3', pbc=False) 88 C4H9 [1 1 1 1 1 1 1 1 1 6 6 6 6]
140 Atoms(symbols='C2ClOH3', pbc=False) 89 C4NH5 [1 1 1 1 1 6 6 6 6 7]
140 Atoms(symbols='C2ClOH3', pbc=False) 90 C5H8 [1

143 Atoms(symbols='C2ClH3', pbc=False) 23 CHF3 [1 6 9 9 9]
143 Atoms(symbols='C2ClH3', pbc=False) 24 CHO [1 6 8]
143 Atoms(symbols='C2ClH3', pbc=False) 25 CN [6 7]
143 Atoms(symbols='C2ClH3', pbc=False) 26 CNH [1 6 7]
143 Atoms(symbols='C2ClH3', pbc=False) 27 CNH3O2 [1 1 1 6 7 8 8]
143 Atoms(symbols='C2ClH3', pbc=False) 28 CNH3O2 [1 1 1 6 7 8 8]
143 Atoms(symbols='C2ClH3', pbc=False) 29 CNH5 [1 1 1 1 1 6 7]
143 Atoms(symbols='C2ClH3', pbc=False) 30 C2Cl4 [ 6  6 17 17 17 17]
143 Atoms(symbols='C2ClH3', pbc=False) 31 C2F4 [6 6 9 9 9 9]
143 Atoms(symbols='C2ClH3', pbc=False) 32 C2H [1 6 6]
143 Atoms(symbols='C2ClH3', pbc=False) 33 C2H2 [1 1 6 6]
143 Atoms(symbols='C2ClH3', pbc=False) 34 C2H2O2 [1 1 6 6 8 8]
143 Atoms(symbols='C2ClH3', pbc=False) 35 C2H2O [1 1 6 6 8]
143 Atoms(symbols='C2ClH3', pbc=False) 36 C2H3Cl [ 1  1  1  6  6 17]
143 Atoms(symbols='C2ClH3', pbc=False) 37 C2H3 [1 1 1 6 6]
143 Atoms(symbols='C2ClH3', pbc=False) 38 C2H3F [1 1 1 6 6 9]
143 Atoms(symbols='C2ClH3', pbc=Fals

146 Atoms(symbols='CCl2H2', pbc=False) 20 CH4O [1 1 1 1 6 8]
146 Atoms(symbols='CCl2H2', pbc=False) 21 CH4S [ 1  1  1  1  6 16]
146 Atoms(symbols='CCl2H2', pbc=False) 22 CHCl3 [ 1  6 17 17 17]
146 Atoms(symbols='CCl2H2', pbc=False) 23 CHF3 [1 6 9 9 9]
146 Atoms(symbols='CCl2H2', pbc=False) 24 CHO [1 6 8]
146 Atoms(symbols='CCl2H2', pbc=False) 25 CN [6 7]
146 Atoms(symbols='CCl2H2', pbc=False) 26 CNH [1 6 7]
146 Atoms(symbols='CCl2H2', pbc=False) 27 CNH3O2 [1 1 1 6 7 8 8]
146 Atoms(symbols='CCl2H2', pbc=False) 28 CNH3O2 [1 1 1 6 7 8 8]
146 Atoms(symbols='CCl2H2', pbc=False) 29 CNH5 [1 1 1 1 1 6 7]
146 Atoms(symbols='CCl2H2', pbc=False) 30 C2Cl4 [ 6  6 17 17 17 17]
146 Atoms(symbols='CCl2H2', pbc=False) 31 C2F4 [6 6 9 9 9 9]
146 Atoms(symbols='CCl2H2', pbc=False) 32 C2H [1 6 6]
146 Atoms(symbols='CCl2H2', pbc=False) 33 C2H2 [1 1 6 6]
146 Atoms(symbols='CCl2H2', pbc=False) 34 C2H2O2 [1 1 6 6 8 8]
146 Atoms(symbols='CCl2H2', pbc=False) 35 C2H2O [1 1 6 6 8]
146 Atoms(symbols='CCl2H2', pbc=F

In [13]:
df.to_csv('../data/haunschild_g2/g2_97.csv')

In [14]:
df = pd.read_csv('../data/haunschild_g2/g2_97.csv',index_col=0)

In [15]:
df['xyz_idx'] = df.xyz_idx.apply(lambda x: int(x.replace('[','').replace(']','')))

ValueError: invalid literal for int() with base 10: '105, 106'

In [45]:
df[df.Formula == 'H2'].E*kJpmol

108    4.744037
Name: E, dtype: object

In [ ]:
df = df.set_index('xyz_idx')

In [ ]:
df.to_csv('g2_97.csv')

In [ ]:
df

In [ ]:
from ase.calculators.singlepoint import SinglePointCalculator

In [43]:
from ase.units import kJ,mol
kJpmol = kJ/mol

In [ ]:
atoms = []
for a_idx, a in enumerate(read('g2_97.xyz',':')):
    a.calc = SinglePointCalculator(a)
    a.calc.results['energy'] = df.loc[a_idx,'E'] * kJpmol
    a.info['name'] = df.loc[a_idx,'Name']
    atoms.append(a)

In [ ]:
write('g2_97.traj', atoms)

In [ ]:
import PyPDF2 as pdf

# importing required modules 
import PyPDF2 
  
# creating a pdf file object 
pdfFileObj = open('papers/haunschildsupplementalmaterial.pdf', 'rb') 
  
# creating a pdf reader object 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
  
# printing number of pages in pdf file 
print(pdfReader.numPages) 
  
text = ''
# creating a page object 
for p in range(2,45):
    pageObj = pdfReader.getPage(p) 
  
    # extracting text from page 
    page = pageObj.extractText()
    text += page
# closing the pdf file object 
pdfFileObj.close() 


lines = text.split()


coords = []
new_mol = True
cnt = 2
name = ''
for i,l in enumerate(lines):
    if len(l) < 3 and l.lower() == l and not l.isnumeric():
        if cnt > 1:
            name = lines[i-2]
        coords.append([l,name, lines[i-1]])
        cnt = 0
    else:
        cnt += 1

def coords_from_str(coords):
    try:
        
        c = coords.replace(' ','')
        c = c.split('.')
        x = c[0] +'.'+ c[1][:14]
        y = c[1][14:] +'.'+ c[2][:14]
        z = c[2][14:] +'.'+ c[3]
        x,y,z = float(x),float(y),float(z)
        return x,y,z
    except Exception:
        return coords
    
coords = [[c[0].upper(),c[1],coords_from_str(c[2])] for c in coords]

In [ ]:
text = ''
mol = ''
cont = [] 
def flush(cont):
    t = ''
    try:
        if cont:
            t = """{} \n {} \n""".format(len(cont),cont[0][1])
            for c in cont:
                t += "{}\t{}\t{}\t{} \n".format(c[0], *c[2])
    except Exception:
        pass
    return t

for c in coords:
    if c[1] != mol:
        mol = c[1]
        text += flush(cont)
        cont = []
    cont.append(c) 

In [ ]:
with open('haunschild_coords.xyz','w') as file:
    file.write(text)